In [1]:

!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium --yes 
!conda install -c conda-forge pyquery --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.19.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  24.38 MB/s
geopy-1.19.0-p 100% |################################| Time: 0:00:00  10.66 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.8.3-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  10.24 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.24 MB/s
vincent-0.4.4- 100% |###################

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
wikitables = soup.find_all('table') 
Toronto = pd.read_html(str(wikitables[0]), index_col=None, header=0)[0]
Toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [3]:

# TO VERIFY THAT BOTH METHODS PRODUCE SIMILAR DIMENSIONS
Toronto.shape

(288, 3)

In [4]:

import requests
import numpy as np
import pandas as pd
from pandas.io.html import read_html

# Define the wiki page url var
WIKI_URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# Issue HTTP request to get the URL content
req = requests.get(WIKI_URL)
# Use pandas read_html to read in the content
wikitables = read_html(WIKI_URL, index_col=None, header=0, attrs={"class":["sortable","wikitable"]})
# Get pandas dataframe
Toronto = wikitables[0]
Toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
# TO VERIFY THAT BOTH METHODS PRODUCE SIMILAR DIMENSIONS
Toronto.shape

(288, 3)

In [6]:

# Empty entries to np.nan to drop them in the next step
Toronto['Borough'].replace('', np.nan, inplace=True)
# Drop np.nan to remove rows not containing meaningful data
Toronto.dropna(subset=['Borough'], inplace=True)
# Leave behind rows containing 'Not assigned' in 'Borough'
Toronto = Toronto[Toronto['Borough'] != 'Not assigned']

In [7]:
# Iterate over the dataframe and fix 'Not assigned' for column 'Neighborhood'
for i, _ in Toronto.iterrows():
    if Toronto.loc[i]['Neighbourhood'] == 'Not assigned': Toronto.loc[i]['Neighborhood'] = Toronto.loc[i]['Borough']

In [8]:
# Check datafame shape
Toronto.shape

(211, 3)

In [9]:
# Print the number of rows in the dataframe
print('Number of rows in Toronto dataframe: {}'.format(Toronto.shape[0]))

Number of rows in Toronto dataframe: 211


In [10]:
!conda install -c conda-forge geocoder --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geocoder:   1.38.1-py_0  conda-forge
    orderedset: 2.0-py35_0   conda-forge
    ratelim:    0.1.6-py35_0 conda-forge

orderedset-2.0 100% |################################| Time: 0:00:00  12.97 MB/s
ratelim-0.1.6- 100% |################################| Time: 0:00:00  13.86 MB/s
geocoder-1.38. 100% |################################| Time: 0:00:00  48.68 MB/s


In [11]:
TPS = Toronto['Postcode'].unique()
len(TPS)

103

In [12]:
!wget -q --no-check-certificate -O 'latitude.pickle' 'https://docs.google.com/uc?export=download&id=1PdEOkPErrpBtDgSlDwczIv_KLlpY-YcO'
!wget -q --no-check-certificate -O 'longitude.pickle' 'https://docs.google.com/uc?export=download&id=1XujA04dCARQnlxu-X2ItOVcYQz0MMQh9'

In [13]:
!ls -l *.pickle

-rw-r----- 1 dsxuser dsxuser 1965 Apr 24 09:39 latitude.pickle
-rw-r----- 1 dsxuser dsxuser 1965 Apr 24 09:39 longitude.pickle


In [14]:
import pickle

with open('latitude.pickle', 'rb') as flat:
    latitude = pickle.load(flat)
with open('longitude.pickle', 'rb') as flon:
    longitude = pickle.load(flon)

In [15]:
import geocoder
import time

for postcode in TPS:
    # When offline cache is available use it to avoid Geocoder Google API throttling
    if postcode in latitude.keys(): 
        continue
    while True:
        g = geocoder.google('{}, Toronto, Ontario'.format(postcode))
        lat_lng_coords = g.latlng
        if lat_lng_coords == None:
            print('Trottled response to {}'.format(postcode))
            time.sleep(5)
            continue
        break
    
    if lat_lng_coords != None:
        latitude[postcode] = lat_lng_coords[0]
        longitude[postcode] = lat_lng_coords[1]
print('Successfully populated geo locations')

Successfully populated geo locations


In [16]:
lat = []
lon = []
for i, _ in Toronto.iterrows():
    lat.append(latitude[Toronto.loc[i]['Postcode']])
    lon.append(longitude[Toronto.loc[i]['Postcode']])

In [17]:
Toronto = Toronto.assign(Latitude = lat, Longitude=lon)
Toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
5,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
6,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [18]:

Toronto = Toronto.reset_index(drop=True)
Toronto.to_csv('Toronto.csv')
Toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [19]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [20]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
T_lat = location.latitude
T_lon = location.longitude
print('The geograpical coordinate of Toronto, ON, Canada are {}, {}.'.format(T_lat, T_lon))

The geograpical coordinate of Toronto, ON, Canada are 43.653963, -79.387207.


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


In [22]:

CLIENT_ID = '0C4ANZOALOMRDOJZU2BUYUKB2LOF3N5ADH4A3WJ3UDS3YIJD'
CLIENT_SECRET = 'MIJAG0GZX3JF2RXLGPCOOH25QZ3NBF2MLFYQJUG2DD5LCNM3'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0C4ANZOALOMRDOJZU2BUYUKB2LOF3N5ADH4A3WJ3UDS3YIJD
CLIENT_SECRET:MIJAG0GZX3JF2RXLGPCOOH25QZ3NBF2MLFYQJUG2DD5LCNM3


In [23]:
HighPark = Toronto[Toronto['Borough'] == 'West Toronto']
HighPark.reset_index(drop=True)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M6H,West Toronto,Dovercourt Village,43.669005,-79.442259
1,M6H,West Toronto,Dufferin,43.669005,-79.442259
2,M6J,West Toronto,Little Portugal,43.647927,-79.419750
3,M6J,West Toronto,Trinity,43.647927,-79.419750
4,M6K,West Toronto,Brockton,43.636847,-79.428191
5,M6K,West Toronto,Exhibition Place,43.636847,-79.428191
6,M6K,West Toronto,Parkdale Village,43.636847,-79.428191
7,M6P,West Toronto,High Park,43.661608,-79.464763
8,M6P,West Toronto,The Junction South,43.661608,-79.464763
9,M6R,West Toronto,Parkdale,43.648960,-79.456325


In [25]:

radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    HP_lat, 
    HP_lon, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=0C4ANZOALOMRDOJZU2BUYUKB2LOF3N5ADH4A3WJ3UDS3YIJD&client_secret=MIJAG0GZX3JF2RXLGPCOOH25QZ3NBF2MLFYQJUG2DD5LCNM3&v=20180605&ll=43.6690051,-79.4422593&radius=500&limit=100'

In [26]:
results = requests.get(url).json()
# results
if results != None:
    print("Request successfully processed")

Request successfully processed


In [27]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Greater Good Bar,Bar,43.669409,-79.439267
1,Parallel,Middle Eastern Restaurant,43.669516,-79.438728
2,Happy Bakery & Pastries,Bakery,43.667050,-79.441791
3,FreshCo,Supermarket,43.667918,-79.440754
4,Planet Fitness Toronto Galleria,Gym / Fitness Center,43.667588,-79.442574


In [29]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

20 venues were returned by Foursquare.


In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:

# type your answer here
HP_venues = getNearbyVenues(names=HighPark['Neighbourhood'],
                                   latitudes=HighPark['Latitude'],
                                   longitudes=HighPark['Longitude']
                                  )

Dovercourt Village
Dufferin
Little Portugal
Trinity
Brockton
Exhibition Place
Parkdale Village
High Park
The Junction South
Parkdale
Roncesvalles
Runnymede
Swansea


In [32]:

print(HP_venues.shape)
HP_venues.head()

(390, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Dovercourt Village,43.669005,-79.442259,The Greater Good Bar,43.669409,-79.439267,Bar
1,Dovercourt Village,43.669005,-79.442259,Parallel,43.669516,-79.438728,Middle Eastern Restaurant
2,Dovercourt Village,43.669005,-79.442259,Happy Bakery & Pastries,43.667050,-79.441791,Bakery
3,Dovercourt Village,43.669005,-79.442259,FreshCo,43.667918,-79.440754,Supermarket
4,Dovercourt Village,43.669005,-79.442259,Planet Fitness Toronto Galleria,43.667588,-79.442574,Gym / Fitness Center


In [33]:
HP_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Brockton,22,22,22,22,22,22
Dovercourt Village,20,20,20,20,20,20
Dufferin,20,20,20,20,20,20
Exhibition Place,22,22,22,22,22,22
High Park,24,24,24,24,24,24
Little Portugal,63,63,63,63,63,63
Parkdale,16,16,16,16,16,16
Parkdale Village,22,22,22,22,22,22
Roncesvalles,16,16,16,16,16,16


In [34]:
print('There are {} uniques categories.'.format(len(HP_venues['Venue Category'].unique())))

There are 92 uniques categories.


In [35]:

# one hot encoding
HP_onehot = pd.get_dummies(HP_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
HP_onehot['Neighborhood'] = HP_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [HP_onehot.columns[-1]] + list(HP_onehot.columns[:-1])
HP_onehot = HP_onehot[fixed_columns]

HP_onehot.head()

,Neighborhood,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Climbing Gym,Cocktail Bar,Coffee Shop,Convenience Store,Cuban Restaurant,Cupcake Shop,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Mac & Cheese Joint,Malay Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Music Venue,New American Restaurant,Nightclub,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Pool,Post Office,Pub,Record Shop,Restaurant,Salon / Barbershop,Sandwich Place,Smoothie Shop,Southern / Soul Food Restaurant,Speakeasy,Sports Bar,Stadium,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Dovercourt Village,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Dovercourt Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Dovercourt Village,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Dovercourt Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,Dovercourt Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [36]:

HP_onehot.shape

(390, 93)

In [37]:

HP_grouped = HP_onehot.groupby('Neighborhood').mean().reset_index()
HP_grouped

,Neighborhood,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Climbing Gym,Cocktail Bar,Coffee Shop,Convenience Store,Cuban Restaurant,Cupcake Shop,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Mac & Cheese Joint,Malay Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Music Venue,New American Restaurant,Nightclub,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Pool,Post Office,Pub,Record Shop,Restaurant,Salon / Barbershop,Sandwich Place,Smoothie Shop,Southern / Soul Food Restaurant,Speakeasy,Sports Bar,Stadium,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Brockton,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.045455,0.000000,0.000000,0.00,0.090909,0.000000,0.0000,0.045455,0.090909,0.000000,0.045455,0.045455,0.000000,0.090909,0.045455,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.045455,0.045455,0.045455,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.045455,0.000000,0.045455,0.045455,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Dovercourt Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.100000,0.0500,0.050000,0.000000,0.000000,0.05,0.000000,0.050000,0.0000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.000000,0.0000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.000000,0.000000,0.000000,0.000000,0.05,0.000000,0.0000,0.050000,0.000000,0.000000,0.050000,0.000000,0.000000,0.100000,0.000000,0.05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Dufferin,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.100000,0.0500,0.050000,0.000000,0.000000,0.05,0.000000,0.050000,0.0000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.000000,0.0000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.000000,0.000000,0.000000,0.000000,0.05,0.000000,0.0000,0.050000,0.000000,0.000000,0.050000,0.000000,0.000000,0.100000,0.000000,0.05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Exhibition Place,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.045455,0.000000,0.000000,0.00,0.090909,0.000000,0.0000,0.045455,0.090909,0.000000,0.045455,0.045455,0.000000,0.090909,0.045455,0.0

In [38]:
HP_grouped.shape

(13, 93)

In [39]:

num_top_venues = 5

for hood in HP_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = HP_grouped[HP_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Brockton----
            venue  freq
0  Breakfast Spot  0.09
1     Coffee Shop  0.09
2            Café  0.09
3      Restaurant  0.05
4       Pet Store  0.05


----Dovercourt Village----
                       venue  freq
0                   Pharmacy  0.10
1                     Bakery  0.10
2                Supermarket  0.10
3  Middle Eastern Restaurant  0.05
4       Brazilian Restaurant  0.05


----Dufferin----
                       venue  freq
0                   Pharmacy  0.10
1                     Bakery  0.10
2                Supermarket  0.10
3  Middle Eastern Restaurant  0.05
4       Brazilian Restaurant  0.05


----Exhibition Place----
            venue  freq
0  Breakfast Spot  0.09
1     Coffee Shop  0.09
2            Café  0.09
3      Restaurant  0.05
4       Pet Store  0.05


----High Park----
                venue  freq
0                Café  0.08
1  Mexican Restaurant  0.08
2                 Bar  0.08
3        Antique Shop  0.04
4  Italian Restaurant  0.04


----Little

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = HP_grouped['Neighborhood']

for ind in np.arange(HP_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(HP_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brockton,Coffee Shop,Café,Breakfast Spot,Gym / Fitness Center,Restaurant,Grocery Store,Gym,Furniture / Home Store,Italian Restaurant,Falafel Restaurant
1,Dovercourt Village,Pharmacy,Bakery,Supermarket,Gym / Fitness Center,Brazilian Restaurant,Middle Eastern Restaurant,Park,Discount Store,Pool,Liquor Store
2,Dufferin,Pharmacy,Bakery,Supermarket,Gym / Fitness Center,Brazilian Restaurant,Middle Eastern Restaurant,Park,Discount Store,Pool,Liquor Store
3,Exhibition Place,Coffee Shop,Café,Breakfast Spot,Gym / Fitness Center,Restaurant,Grocery Store,Gym,Furniture / Home Store,Italian Restaurant,Falafel Restaurant
4,High Park,Mexican Restaurant,Café,Bar,Antique Shop,Speakeasy,Diner,Park,Cajun / Creole Restaurant,Fast Food Restaurant,Flea Market
5,Little Portugal,Bar,Asian Restaurant,Coffee Shop,Café,Vietnamese Restaurant,Pizza Place,New American Restaurant,Bakery,Restaurant,Men's Store
6,Parkdale,Breakfast Spot,Gift Shop,Bookstore,Cuban Restaurant,Movie Theater,Dog Run,Eastern European Restaurant,Restaurant,Burger Joint,Dessert Shop
7,Parkdale Village,Coffee Shop,Café,Breakfast Spot,Gym / Fitness Center,Restaurant,Grocery Store,Gym,Furniture / Home Store,Italian Restaurant,Falafel Restaurant
8,Roncesvalles,Breakfast Spot,Gift Shop,Bookstore,Cuban Restaurant,Movie Theater,Dog Run,Eastern European Restaurant,Restaurant,Burger Joint,Dessert Shop
9,Runnymede,Coffee Shop,Café,Pizza Place,Italian Restaurant,Sushi Restaurant,Burrito Place,Park,French Restaurant,Dessert Shop,Latin American Restaurant


In [42]:
# set number of clusters
kclusters = 5

HP_grouped_clustering = HP_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(HP_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 3, 3, 1, 4, 0, 2, 1, 2, 0], dtype=int32)

In [43]:
HP_merged = HighPark

# add clustering labels
HP_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
HP_merged = HP_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

HP_merged.head() # check the last columns!

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,M6H,West Toronto,Dovercourt Village,43.669005,-79.442259,1,Pharmacy,Bakery,Supermarket,Gym / Fitness Center,Brazilian Restaurant,Middle Eastern Restaurant,Park,Discount Store,Pool,Liquor Store
53,M6H,West Toronto,Dufferin,43.669005,-79.442259,3,Pharmacy,Bakery,Supermarket,Gym / Fitness Center,Brazilian Restaurant,Middle Eastern Restaurant,Park,Discount Store,Pool,Liquor Store
64,M6J,West Toronto,Little Portugal,43.647927,-79.419750,3,Bar,Asian Restaurant,Coffee Shop,Café,Vietnamese Restaurant,Pizza Place,New American Restaurant,Bakery,Restaurant,Men's Store
65,M6J,West Toronto,Trinity,43.647927,-79.419750,1,Bar,Asian Restaurant,Coffee Shop,Café,Vietnamese Restaurant,Pizza Place,New American Restaurant,Bakery,Restaurant,Men's Store
76,M6K,West Toronto,Brockton,43.636847,-79.428191,4,Coffee Shop,Café,Breakfast Spot,Gym / Fitness Center,Restaurant,Grocery Store,Gym,Furniture / Home Store,Italian Restaurant,Falafel Restaurant


In [45]:
HP_merged.loc[HP_merged['Cluster Labels'] == 0, HP_merged.columns[[1] + list(range(5, HP_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
77,West Toronto,0,Coffee Shop,Café,Breakfast Spot,Gym / Fitness Center,Restaurant,Grocery Store,Gym,Furniture / Home Store,Italian Restaurant,Falafel Restaurant
134,West Toronto,0,Breakfast Spot,Gift Shop,Bookstore,Cuban Restaurant,Movie Theater,Dog Run,Eastern European Restaurant,Restaurant,Burger Joint,Dessert Shop
135,West Toronto,0,Breakfast Spot,Gift Shop,Bookstore,Cuban Restaurant,Movie Theater,Dog Run,Eastern European Restaurant,Restaurant,Burger Joint,Dessert Shop
146,West Toronto,0,Coffee Shop,Café,Pizza Place,Italian Restaurant,Sushi Restaurant,Burrito Place,Park,French Restaurant,Dessert Shop,Latin American Restaurant


In [46]:

HP_merged.loc[HP_merged['Cluster Labels'] == 1, HP_merged.columns[[1] + list(range(5, HP_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,West Toronto,1,Pharmacy,Bakery,Supermarket,Gym / Fitness Center,Brazilian Restaurant,Middle Eastern Restaurant,Park,Discount Store,Pool,Liquor Store
65,West Toronto,1,Bar,Asian Restaurant,Coffee Shop,Café,Vietnamese Restaurant,Pizza Place,New American Restaurant,Bakery,Restaurant,Men's Store
124,West Toronto,1,Mexican Restaurant,Café,Bar,Antique Shop,Speakeasy,Diner,Park,Cajun / Creole Restaurant,Fast Food Restaurant,Flea Market


In [47]:

HP_merged.loc[HP_merged['Cluster Labels'] == 3, HP_merged.columns[[1] + list(range(5, HP_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,West Toronto,3,Pharmacy,Bakery,Supermarket,Gym / Fitness Center,Brazilian Restaurant,Middle Eastern Restaurant,Park,Discount Store,Pool,Liquor Store
64,West Toronto,3,Bar,Asian Restaurant,Coffee Shop,Café,Vietnamese Restaurant,Pizza Place,New American Restaurant,Bakery,Restaurant,Men's Store


In [48]:
HP_merged.loc[HP_merged['Cluster Labels'] == 4, HP_merged.columns[[1] + list(range(5, HP_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
76,West Toronto,4,Coffee Shop,Café,Breakfast Spot,Gym / Fitness Center,Restaurant,Grocery Store,Gym,Furniture / Home Store,Italian Restaurant,Falafel Restaurant
145,West Toronto,4,Coffee Shop,Café,Pizza Place,Italian Restaurant,Sushi Restaurant,Burrito Place,Park,French Restaurant,Dessert Shop,Latin American Restaurant


Thank you for reviewing my work!